## Inject Bugs & Run Experiment

![Inject Bugs & Run Experiment](./pic/RL_Testing.drawio.png)

#### Frozenlake Testbed Experiment

- Find log files in RLTestig/logs/frozenlake

In [ ]:
import frozenlake.testing_SB3_Frozenlake as Frozenlake_Experiment

# initialize bug_version_list
bug_version_list = [
    [],
    # [0],
    # [1],
    # [2],
    # [3],
    # [4],
    #
    # [6],
    # [7],
    # [8],
    # [9],
    # [10],
]

Frozenlake_Experiment.main(bug_version_list=bug_version_list, rounds=25, epochs=300, model_type='dqn')


#### Mountaincar Testbed (continous) Experiment

- Find log files in RLTestig/logs/mountaincar

In [29]:
import mountaincar.testing_SB3_Mountaincar as Mountaincar_Experiment

bug_version_list = [
    [],
    # [0],
    # [1],
    # [2],
    # [3],
    # [4],
    # [5],
    # [6],
    # [7],
    # [8],
    # [9],
    # [10],
]

Mountaincar_Experiment.main(bug_version_list=bug_version_list, rounds=10, epochs=300, model_type='ppo')


(array([-0.43373287,  0.        ], dtype=float32), {})
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Obtaining file:///Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for stable_baselines3 (pyproject.toml): started
  Building editable for stable_baselines3 (pyproject.toml): finished with status 'done'
  Created wheel for stable_baselines3: filename=stable_baselines3-2.3.0a3-0.editable-py3-none-any.whl size=6132 sha25

/Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 16`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 16
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=16 and n_envs=1)
  warnings.warn(
/Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new/stable_baselines3/common/vec_env/base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")
/Users/zhangshiyu/opt/anaconda3/envs/stable-baselines3-testing-new/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_state_action_pairs to get variables from other wrappers is deprecated and will be removed in v1.0, to 

round: 1----
creating new model


## Validation Part

In [26]:
import log_parser
import pandas as pd
from scipy.stats import linregress
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import config_parser
from scipy.stats import binomtest


root_dir = config_parser.parserConfig()['root_dir']


def linelar_regression(data, title = 'Data Trend Analysis', show=False):

    # 将数据转换成Pandas Series对象
    series = pd.Series(data)

    # 计算简单移动平均(SMA)和指数移动平均(EMA)
    sma = series.rolling(window=3).mean()
    ema = series.ewm(span=3, adjust=False).mean()

    # 使用线性回归判断趋势
    slope, intercept, r_value, p_value, std_err = linregress(range(len(data)), data)
    
    if show:
        # 绘制数据和趋势线
        plt.figure(figsize=(12, 6))
        plt.plot(data, label='Original Data', marker='o')
        plt.plot(range(len(data)), intercept + slope * np.asarray(range(len(data))), 'r', label=f'Trend Line: slope={slope:.2f}')
        plt.legend()
        plt.title(title)
        plt.show()

    # 输出线性回归结果
    # print(f"Slope of trend line: {slope:.2f}")

    # 如果斜率显著小于0，我们可以认为不存在上升趋势
    if slope < 0:
        return 'false'
    else:
        return 'true'
    

def test_true_proportion(num_samples, num_true, sig_level=0.05, expected_prob=0.95):
    """
    对观察到的 'true' 样本数进行二项检验。

    :param num_samples: 样本总数
    :param num_true: 观察到的 'true' 样本数
    :param sig_level: 显著性水平（默认为 0.05）
    :param expected_prob: 预期成功概率（默认为 0.5）
    :return: p 值和是否显著
    """
    # 计算二项检验的 p 值
    p_value = binomtest(k=num_true, n=num_samples, p=expected_prob, alternative='greater')

    # 判断是否显著
    is_significant = p_value < sig_level

    return p_value, is_significant


def bin_test_frozenlake(bug_free_log_path = Path(os.path.join(root_dir, 'RLTesting', 'logs', 'Frozenlake', 'dqn', '[]')), show_fig=False):
    # 使用 Path.rglob() 获取所有子文件
    # '**/*' 表示匹配所有的文件和文件夹, 如果只想匹配文件, 使用 '**/*.*'
    bug_free_log_list = [file for file in bug_free_log_path.rglob('*') if file.is_file()]
    print(bug_free_log_list)

    result = []
    for path in bug_free_log_list:
        # print(log_parser.parse_log_file(path))
        data = log_parser.parse_log_file_fuzzy(path)
        if len(data)> 0:
            temp = linelar_regression(data, title=path, show=show_fig)
        result.append(temp)

    p_value = binomtest(result.count('true'), len(result), alternative='greater')
    print(p_value)

    
    
def bin_test_mountaincar(bug_free_log_path = Path(os.path.join(root_dir, 'RLTesting', 'logs', 'Mountaincar', 'sac', '[]')), show_fig=False):
    # 使用 Path.rglob() 获取所有子文件
    # '**/*' 表示匹配所有的文件和文件夹, 如果只想匹配文件, 使用 '**/*.*'
    bug_free_log_list = [file for file in bug_free_log_path.rglob('*') if file.is_file()]

    print(bug_free_log_list)

    # accuracy_list = []
    # 打印所有文件路径
    result = []
    for path in bug_free_log_list:
        data = log_parser.parse_mountaincar_log_file(path)
        if len(data) > 0:
            temp = linelar_regression(data, title=path, show=show_fig)
        result.append(temp)
    
    
    '''
    在进行二项检验时，零假设（null hypothesis）通常是观察到的成功次数与随机变化相一致，即成功的概率等于 expected_prob。如果实际观察到的成功次数显著高于（或低于）这个期望成功概率，那么您可能会得到一个很小的 p 值，从而拒绝零假设。
    '''
    p_value = binomtest(result.count('true'), len(result), alternative='greater')
    print(p_value)



In [28]:
bin_test_frozenlake(bug_free_log_path = Path(os.path.join(root_dir, 'RLTesting', 'logs', 'Frozenlake', 'dqn', '[]')), show_fig=False)
bin_test_mountaincar(bug_free_log_path = Path(os.path.join(root_dir, 'RLTesting', 'logs', 'Mountaincar', 'a2c', '[]')), show_fig=False)

[PosixPath('/Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new/RLTesting/logs/Frozenlake/dqn/[]/time_2024-03-01[]round_10'), PosixPath('/Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new/RLTesting/logs/Frozenlake/dqn/[]/time_2024-03-01[]round_17'), PosixPath('/Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new/RLTesting/logs/Frozenlake/dqn/[]/time_2024-03-01[]round_21'), PosixPath('/Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new/RLTesting/logs/Frozenlake/dqn/[]/time_2024-03-01[]round_19'), PosixPath('/Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new/RLTesting/logs/Frozenlake/dqn/[]/time_2024-03-01[]round_6'), PosixPath('/Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new/RLTesting/logs/Frozenlake/dqn/[]/time_2024-03-01[]round_1'), PosixPath('/Users/zhangshiyu/Documents/GitHub/stable-baselines3-testing-new/RLTesting/logs/Frozenlake/dqn/[]/time_2024-03-01[]round_8'), PosixPath('/Users/zhangshiyu/Documen